<a href="https://colab.research.google.com/github/SeoMinJong/1day-1study/blob/master/credit_my_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터를 가져와서 기본적인 설정과 결측치 제거 및 column들 데이터 파악

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option("precision",2) # precision 소숫점 자리수를 두 자리까지만 출력하는 pandas option

In [17]:
df = pd.read_csv('/content/drive/MyDrive/BigData/credit/data_credit.csv')

In [18]:
df.head(3)

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   store_id      int64  
 1   date          object 
 2   time          object 
 3   card_id       object 
 4   amount        int64  
 5   installments  float64
 6   days_of_week  int64  
 7   holyday       int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 205.2+ MB


In [20]:
# 결측치가 있다. 결측치가 얼마나 있는지 파악해봐야 한다.
null_df = df.isnull()
null_df.sum()
# 결측치 확인 결과 installments만 결측치가 있는 것으로 판별

store_id              0
date                  0
time                  0
card_id               0
amount                0
installments    3345936
days_of_week          0
holyday               0
dtype: int64

In [21]:
# null값을 제외한 다른 값들은 어떤 것이 있고 얼마나 있는지 확인할 수 있다.
df.installments.value_counts()

3.0     10686
2.0      3431
5.0      1220
6.0       647
10.0      252
4.0       209
12.0      206
24.0       96
8.0        32
7.0        28
9.0        25
18.0       11
20.0        6
36.0        5
22.0        4
15.0        2
Name: installments, dtype: int64

In [34]:
# df에서 null값을 fillna를 사용해서 결측치를 1로 대체해준다.
# df에서 null값이 있는 colunm은 installments를 가져와서 그대로 installments에 넣어줘서 결측치를 제거해 줬다.
# 그 후 installments의 개수를 알기 위해 value_counts를 사용해서 총 개수들을 알수있다.
df.installments = df.installments.fillna(value=1)
df.installments.value_counts()

1.0     3345936
3.0       10686
2.0        3431
5.0        1220
6.0         647
10.0        252
4.0         209
12.0        206
24.0         96
8.0          32
7.0          28
9.0          25
18.0         11
20.0          6
36.0          5
22.0          4
15.0          2
Name: installments, dtype: int64

In [36]:
# data와 time을 그대로 추출해서 쓰기 어렵기 때문에 data에서 month와 time에서 hour을 뽑아서 새로운 column을 만들어준다.
df['month'] = df.date.apply( lambda x : x[5:7] ) # month에 대한 column을 만든다.
df['hour'] = df.time.apply( lambda x : x[:2]) # hour에 대한 columns을 만든다.

In [37]:
df.head(5)

,store_id,date,time,card_id,amount,installments,days_of_week,holyday,month,hour
0,0,2016-12-14,18:05:31,d297bba73f,5,1.0,2,0,12,18
1,0,2016-12-14,18:05:54,d297bba73f,-5,1.0,2,0,12,18
2,0,2016-12-19,12:42:31,0880849c05,144,1.0,0,0,12,12
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,1.0,0,0,12,12
4,0,2016-12-19,13:31:08,7ad237eed0,24,1.0,0,0,12,13


In [41]:
# 필요한 month와 hour를 빼냈으니 필요없는 date와 time을 삭제해준다.
df = df.drop(columns=['date','time'])
df.head(3)

,store_id,card_id,amount,installments,days_of_week,holyday,month,hour
0,0,d297bba73f,5,1.0,2,0,12,18
1,0,d297bba73f,-5,1.0,2,0,12,18
2,0,0880849c05,144,1.0,0,0,12,12


# 완성된 df를 분석 시작

In [23]:
df.head(3)

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,1.0,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,1.0,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,1.0,0,0
